To this point, the seed labeling subset has been extracted using reservoir sampling, creating a random set of 10,000 examples. Now, this subset, which has been written to seed_labeling_unlabeled.csv, will be labeled and updated with the regex functions found in diet_classificaion.py